In [6]:
from gamestate import GameState, start_ai_only_game
from models.RandomModel import RandomModel
from models.HeuristicModel import HeuristicModel
from models.DeepRLModel import DeepRLModel
from trainutil import train_models, test_models
import fsutils as fs

In [2]:
import torch
torch.cuda.is_available()

False

In [3]:
env = GameState()
deep_rl_model = DeepRLModel()
rand_model_1 = RandomModel(min_steps=5, max_steps=10)
rand_model_2 = RandomModel(min_steps=5, max_steps=10)
models = [deep_rl_model, rand_model_1, rand_model_2]

In [4]:
train_models(env, models, episodes=10, steps=3000, print_every=500)
test_models(env, models, steps=3000)

TRAIN mode
[0] [FOOD] Agent0 ate food item 250
[0] [FOOD] Agent0 ate food item 263
[0] [FOOD] Agent0 ate food item 275
[0] [FOOD] Agent1 ate food item 203
[0] [FOOD] Agent2 ate food item 208
[0] [FOOD] Agent2 ate food item 278
[0] [FOOD] Agent2 ate food item 291
[0] [FOOD] Agent2 ate food item 311
----STEP 0 rewards----
Model 0: 2.9799999999999898
Model 1: 0.9799999999999898
Model 2: 3.9799999999999898
[44] [FOOD] Agent2 ate food item 314
[54] [FOOD] Agent0 ate food item 324
[62] [FOOD] Agent0 ate food item 221
[62] [FOOD] Agent1 ate food item 383
[62] [VIRUS] Agent1 ate virus 9
[68] [FOOD] Agent1 ate food item 266
[71] [FOOD] Agent1 ate food item 366
[74] [FOOD] Agent1 ate food item 337
[77] [FOOD] Agent1 ate food item 262
[77] [FOOD] Agent1 ate food item 382
[78] [FOOD] Agent1 ate food item 273
[80] [FOOD] Agent1 ate food item 202
[86] [FOOD] Agent1 ate food item 356
[86] [FOOD] Agent2 ate food item 313
[88] [FOOD] Agent1 ate food item 379
[93] [FOOD] Agent2 ate food item 211
[114] [

[78] [FOOD] Agent1 ate food item 555
[79] [FOOD] Agent1 ate food item 731
[84] [FOOD] Agent2 ate food item 752
[86] [FOOD] Agent2 ate food item 621
[92] [FOOD] Agent1 ate food item 599
[93] [FOOD] Agent0 ate food item 548
[109] [FOOD] Agent1 ate food item 570
[110] [FOOD] Agent0 ate food item 643
[111] [FOOD] Agent1 ate food item 724
[113] [FOOD] Agent0 ate food item 655
[118] [FOOD] Agent2 ate food item 682
[127] [FOOD] Agent2 ate food item 577
[131] [FOOD] Agent1 ate food item 703
[132] [FOOD] Agent1 ate food item 700
[133] [FOOD] Agent2 ate food item 750
[136] [FOOD] Agent1 ate food item 536
[137] [FOOD] Agent2 ate food item 683
[138] [FOOD] Agent1 ate food item 537
[141] [FOOD] Agent1 ate food item 568
[148] [FOOD] Agent1 ate food item 628
[156] [FOOD] Agent0 ate food item 647
[158] [FOOD] Agent0 ate food item 772
[166] [CELL] Agent2 ate one of Agent0's cells
[167] [FOOD] Agent2 ate food item 667
[167] [VIRUS] Agent2 ate virus 18
[168] [CELL] Agent0 ate one of Agent2's cells
[169] 

[238] [FOOD] Agent0 ate food item 1139
[239] [FOOD] Agent0 ate food item 969
[240] [FOOD] Agent0 ate food item 917
[244] [FOOD] Agent0 ate food item 1089
[244] [FOOD] Agent0 ate food item 1117
[247] [FOOD] Agent0 ate food item 951
[248] [FOOD] Agent0 ate food item 1092
[250] [FOOD] Agent0 ate food item 1079
[250] [FOOD] Agent0 ate food item 1081
[259] [FOOD] Agent0 ate food item 887
[259] [FOOD] Agent0 ate food item 926
[262] [FOOD] Agent0 ate food item 920
[263] [FOOD] Agent0 ate food item 1012
[265] [FOOD] Agent0 ate food item 933
[266] [FOOD] Agent0 ate food item 907
[267] [FOOD] Agent0 ate food item 958
[276] [FOOD] Agent0 ate food item 923
[277] [FOOD] Agent0 ate food item 1094
[287] [FOOD] Agent0 ate food item 897
[310] [FOOD] Agent2 ate food item 1076
[311] [FOOD] Agent2 ate food item 1040
[322] [FOOD] Agent2 ate food item 927
[350] [FOOD] Agent0 ate food item 1153
[355] [FOOD] Agent0 ate food item 1115
[357] [FOOD] Agent2 ate food item 1114
[364] [FOOD] Agent2 ate food item 103

[261] [FOOD] Agent0 ate food item 1518
[261] [FOOD] Agent0 ate food item 1681
[268] [FOOD] Agent0 ate food item 1494
[275] [FOOD] Agent0 ate food item 1670
[309] [VIRUS] Agent1 ate virus 35
[311] [VIRUS] Agent2 ate virus 34
[317] [FOOD] Agent2 ate food item 1568
[333] [FOOD] Agent2 ate food item 1666
[344] [FOOD] Agent0 ate food item 1488
[345] [FOOD] Agent0 ate food item 1680
[359] [FOOD] Agent2 ate food item 1473
[361] [FOOD] Agent2 ate food item 1534
[365] [FOOD] Agent0 ate food item 1669
[367] [FOOD] Agent2 ate food item 1524
[390] [FOOD] Agent1 ate food item 1673
[395] [FOOD] Agent2 ate food item 1711
[407] [FOOD] Agent2 ate food item 1594
[445] [FOOD] Agent2 ate food item 1467
[445] [FOOD] Agent2 ate food item 1541
[447] [FOOD] Agent0 ate food item 1600
[447] [FOOD] Agent2 ate food item 1722
[449] [FOOD] Agent0 ate food item 1509
[449] [FOOD] Agent0 ate food item 1725
[450] [FOOD] Agent1 ate food item 1730
[452] [FOOD] Agent1 ate food item 1732
[457] [FOOD] Agent2 ate food item 1

[939] [FOOD] Agent1 ate food item 2050
[946] [FOOD] Agent1 ate food item 1806
[952] [FOOD] Agent1 ate food item 2000
----STEP 1000 rewards----
Model 0: 115.6312665501338
Model 1: 93.73894844342234
Model 2: 2.242043376793333
[1008] [FOOD] Agent1 ate food item 2079
[1044] [FOOD] Agent1 ate food item 1936
[1088] [FOOD] Agent0 ate food item 2080
[1122] [CELL] Agent2 ate one of Agent1's cells
[1123] [CELL] Agent2 ate one of Agent1's cells
[1124] [CELL] Agent2 ate one of Agent1's cells
[1129] [CELL] Agent2 ate one of Agent1's cells
[1140] [CELL] Agent2 ate one of Agent1's cells
[1140] [GAME] Agent1 died! Was eaten by Agent2
[1309] [FOOD] Agent0 ate food item 2070
[1340] [FOOD] Agent0 ate food item 1940
[1398] [FOOD] Agent0 ate food item 1941
[1405] [FOOD] Agent0 ate food item 1843
----STEP 1500 rewards----
Model 0: 111.2679299695505
Model 1: -239.1494816453824
Model 2: 273.6979441033367
[1593] [FOOD] Agent0 ate food item 1866
[1601] [FOOD] Agent0 ate food item 2020
[1629] [FOOD] Agent0 ate f

[6] [FOOD] Agent1 ate food item 2691
[6] [FOOD] Agent1 ate food item 2742
[17] [FOOD] Agent0 ate food item 2656
[22] [FOOD] Agent0 ate food item 2833
[24] [FOOD] Agent0 ate food item 2741
[25] [FOOD] Agent1 ate food item 2843
[31] [FOOD] Agent2 ate food item 2832
[38] [FOOD] Agent2 ate food item 2640
[42] [FOOD] Agent1 ate food item 2809
[45] [FOOD] Agent2 ate food item 2729
[56] [FOOD] Agent2 ate food item 2643
[75] [FOOD] Agent0 ate food item 2785
[91] [FOOD] Agent2 ate food item 2752
[102] [FOOD] Agent0 ate food item 2803
[111] [FOOD] Agent0 ate food item 2818
[117] [FOOD] Agent2 ate food item 2636
[118] [FOOD] Agent0 ate food item 2802
[118] [FOOD] Agent2 ate food item 2823
[119] [FOOD] Agent1 ate food item 2739
[132] [FOOD] Agent1 ate food item 2770
[135] [FOOD] Agent0 ate food item 2796
[156] [FOOD] Agent1 ate food item 2681
[161] [FOOD] Agent2 ate food item 2786
[167] [FOOD] Agent1 ate food item 2679
[186] [FOOD] Agent2 ate food item 2792
[202] [FOOD] Agent2 ate food item 2659
[

/home/ubuntu/cis-522-final-project/agar-py/models/DeepRLModel.py:93: RuntimeWarning: divide by zero encountered in true_divide
  obj_dists_inv_np = 1 / np.sqrt(obj_dists_np)


----STEP 1500 rewards----
Model 0: -18.356764828492175
Model 1: -15.590361579335195
Model 2: -7.485621544925493
[1689] [FOOD] Agent0 ate food item 2874
[1969] [FOOD] Agent0 ate food item 2720
[1986] [FOOD] Agent0 ate food item 2654
[1997] [FOOD] Agent0 ate food item 2860
----STEP 2000 rewards----
Model 0: -23.251459224566332
Model 1: -24.58456434110056
Model 2: -16.87511641737038
[2004] [FOOD] Agent0 ate food item 2856
[2008] [FOOD] Agent0 ate food item 2734
[2010] [FOOD] Agent0 ate food item 2760
[2011] [FOOD] Agent0 ate food item 2761
[2014] [FOOD] Agent0 ate food item 2794
[2045] [FOOD] Agent0 ate food item 2882
[2064] [VIRUS] Agent0 ate virus 62
[2066] [FOOD] Agent0 ate food item 2698
[2066] [FOOD] Agent0 ate food item 2829
[2068] [FOOD] Agent0 ate food item 2745
[2068] [FOOD] Agent0 ate food item 2873
[2073] [FOOD] Agent0 ate food item 2686
[2152] [FOOD] Agent0 ate food item 2660
[2154] [FOOD] Agent0 ate food item 2839
[2155] [FOOD] Agent0 ate food item 2712
[2156] [FOOD] Agent0 a

[1383] [FOOD] Agent2 ate food item 3261
[1394] [FOOD] Agent2 ate food item 3247
[1395] [FOOD] Agent1 ate food item 3267
[1418] [FOOD] Agent2 ate food item 3252
[1435] [FOOD] Agent2 ate food item 3214
----STEP 1500 rewards----
Model 0: 100.01187939410345
Model 1: 114.02503230938053
Model 2: 139.8279260855549
[1591] [FOOD] Agent2 ate food item 3210
[1602] [FOOD] Agent2 ate food item 3181
[1607] [FOOD] Agent1 ate food item 3274
[1645] [FOOD] Agent1 ate food item 3268
[1679] [FOOD] Agent0 ate food item 2955
[1693] [FOOD] Agent1 ate food item 3117
[1702] [FOOD] Agent1 ate food item 2925
[1705] [FOOD] Agent1 ate food item 3011
[1855] [FOOD] Agent0 ate food item 2926
[1855] [FOOD] Agent0 ate food item 2968
----STEP 2000 rewards----
Model 0: 101.3718573650561
Model 1: 107.62517836179185
Model 2: 125.17441722922207
[2082] [CELL] Agent2 ate one of Agent1's cells
[2095] [CELL] Agent2 ate one of Agent1's cells
[2096] [VIRUS] Agent2 ate virus 71
[2232] [FOOD] Agent2 ate food item 3190
----STEP 2500

In [5]:
fs.save_net_to_disk(deep_rl_model.model, "test_drl_5")

NameError: name 'fs' is not defined